In [2]:
import pandas as pd
import numpy as np
from datetime import datetime

FILE_NAME = 'Employee_Data.csv'
CURRENT_DATE = datetime.now()

def analyze_employee_data(file_name, current_date):

    df = pd.read_csv('/Users/kaarunyalakshmanchinthalapudi/Downloads/Employee_Data.csv')
    df.columns = df.columns.str.strip()

    # --- 1. Data Quality Check and Preprocessing ---
    df.drop_duplicates(inplace=True)
    critical_columns = ["EmpID", "StartDate", "EmployeeStatus"]
    df.dropna(subset=critical_columns, inplace=True)

    # Convert Date columns to datetime objects
    df['StartDate'] = pd.to_datetime(df['StartDate'], errors='coerce', dayfirst=False)
    df['ExitDate'] = pd.to_datetime(df['ExitDate'], errors='coerce', dayfirst=False)
    df['DOB'] = pd.to_datetime(df['DOB'], errors='coerce', dayfirst=False)

    df['Performance Score'] = df['Performance Score'].astype(str).str.lower().str.strip()
    perf_map = {
        'fully meets': 3.0, 'exceeds': 4.0, 'needs improvement': 2.0,
        'fully meets expectations': 3.0, 'exceeds expectations': 4.0,
        '90-day meets': 3.0, 'pip': 1.0, 'not meets': 1.0,
        'n/a-hire': np.nan, 'stat': np.nan, 'na': np.nan
    }
    df['Performance_Score_Num'] = df['Performance Score'].replace(perf_map)
    df['Current Employee Rating'] = pd.to_numeric(df['Current Employee Rating'], errors='coerce')


    # --- 2. Calculate the average tenure of employees ---
    df['Actual_ExitDate'] = df['ExitDate'].fillna(current_date)
    df['Tenure_Days'] = (df['Actual_ExitDate'] - df['StartDate']).dt.days
    avg_tenure_years = (df[df['Tenure_Days'] >= 0]['Tenure_Days'].mean() / 365.25).round(2)
    print(f"2. Average Employee Tenure (in years): {avg_tenure_years}")


    # --- 3. Calculate mean, median, and standard deviation for "Performance Score" ---
    perf_stats = df['Performance_Score_Num'].agg(['mean', 'median', 'std']).round(2)
    print(f"\n3. Performance Score Statistics:\n{perf_stats.to_string()}")


    # --- 4. Compute avg Current Employee Rating by DepartmentType ---
    avg_rating_by_dept = df.groupby('DepartmentType')['Current Employee Rating'].mean().sort_values(ascending=False).round(2)
    highest_rated_dept = avg_rating_by_dept.index[0].strip()
    highest_rating = avg_rating_by_dept.iloc[0]
    print(f"\n4. Department with Highest Average Rating: {highest_rated_dept} ({highest_rating})")


    # --- 5. Calculate exited employees by TerminationType and analyze TerminationDescription ---
    exited_employees = df[df['ExitDate'].notna()]
    exit_counts_by_type = exited_employees.groupby('TerminationType')['EmpID'].count().sort_values(ascending=False)
    print(f"\n5. Exit Counts by Termination Type:\n{exit_counts_by_type.head(5).to_string()}")


    # --- 6. Gender and Race Analysis ---
    gender_counts = df['GenderCode'].value_counts()
    race_percentage = (df['RaceDesc'].value_counts(normalize=True) * 100).round(2)
    print(f"\n6. Employee Counts by GenderCode:\n{gender_counts.to_string()}")
    print(f"\n6. Employee Percentage by RaceDesc:\n{race_percentage.to_string()}")


    # --- 7. Calculate avg Performance Score by BusinessUnit (Top 3) ---
    avg_perf_by_bu = df.groupby('BusinessUnit')['Performance_Score_Num'].mean().sort_values(ascending=False).round(2)
    top_3_bu = avg_perf_by_bu.head(3)
    print(f"\n7. Top 3 Business Units by Avg Performance Score:\n{top_3_bu.to_string()}")


    # --- 8. Identify current vs. terminated employees by EmployeeStatus ---
    status_counts = df['EmployeeStatus'].value_counts()
    print(f"\n8. Employee Status Counts:\n{status_counts.to_string()}")


    # --- 9. Total employees and avg Performance Score for each DepartmentType ---
    dept_summary = df.groupby('DepartmentType').agg(
        Total_Employees=('EmpID', 'count'),
        Avg_Performance_Score=('Performance_Score_Num', 'mean')
    ).round(2).sort_values(by='Total_Employees', ascending=False)
    print(f"\n9. Department Type Summary:\n{dept_summary.to_string()}")


    # --- 10. Performance Score Analysis by JobFunctionDescription and PayZone ---
    perf_by_job_payzone = df.groupby(['JobFunctionDescription', 'PayZone'])['Performance_Score_Num'].mean().unstack().round(2).fillna('N/A')
    # Save the full result to a CSV
    perf_by_job_payzone.to_csv("performance_by_job_function_payzone.csv")
    print("\n10. Full table of Avg Performance Score by Job Function and Pay Zone saved to 'performance_by_job_function_payzone.csv'.")


    # --- 11. Analyze avg Performance Score by Supervisor (Top 5) ---
    avg_perf_by_supervisor = df.groupby('Supervisor')['Performance_Score_Num'].mean().sort_values(ascending=False).round(2)
    top_5_supervisors = avg_perf_by_supervisor[avg_perf_by_supervisor.index.notna()].head(5)
    print(f"\n11. Top 5 Supervisors by Average Performance Score:\n{top_5_supervisors.to_string()}")


    # --- 12. Marital Status Analysis ---
    marital_counts = df['MaritalDesc'].value_counts()
    marital_percentage = (df['MaritalDesc'].value_counts(normalize=True) * 100).round(2)
    marital_analysis = pd.DataFrame({'Count': marital_counts, 'Percentage': marital_percentage}).sort_values(by='Count', ascending=False)
    print(f"\n12. Marital Status Counts and Percentages:\n{marital_analysis.to_string()}")


    # --- 13. Age Group Analysis (10-year intervals) ---
    df['Age'] = (current_date - df['DOB']).dt.days / 365.25
    valid_ages = df['Age'].dropna()
    valid_ages = valid_ages[(valid_ages > 18) & (valid_ages < 100)]

    if not valid_ages.empty:
        min_age = int(valid_ages.min() // 10 * 10)
        max_age = int(valid_ages.max() // 10 * 10 + 10)
        age_bins = range(max(0, min_age - 10), max_age + 10, 10)
        age_labels = [f'{i+1}-{i+10}' for i in age_bins[:-1]]
        df['Age_Group'] = pd.cut(df['Age'], bins=age_bins, labels=age_labels, right=False, include_lowest=True)
        age_group_percentage = (df['Age_Group'].value_counts(normalize=True, dropna=False) * 100).sort_index().round(2)
        print(f"\n13. Employee Percentage by 10-Year Age Group:\n{age_group_percentage.to_string()}")


    # --- 14. Analyze avg Performance Score by LocationCode ---
    avg_perf_by_location = df.groupby('LocationCode')['Performance_Score_Num'].mean().sort_values(ascending=False).round(2)
    if not avg_perf_by_location.empty:
        highest_location = avg_perf_by_location.index[0]
        highest_score = avg_perf_by_location.iloc[0]
        lowest_location = avg_perf_by_location.index[-1]
        lowest_score = avg_perf_by_location.iloc[-1]
        print(f"\n14. Location with Highest Avg Performance Score: {highest_location} ({highest_score})")
        print(f"14. Location with Lowest Avg Performance Score: {lowest_location} ({lowest_score})")


In [3]:
import pandas as pd
import numpy as np
from datetime import datetime

# ------------------ Load Dataset ------------------
file_path = "/Users/kaarunyalakshmanchinthalapudi/Downloads/Employee_Data.csv"
df = pd.read_csv(file_path)

# ------------------ 1. Data Quality Check ------------------
df = df.drop_duplicates()
critical_cols = ["EmpID", "StartDate", "EmployeeStatus"]
df = df.dropna(subset=critical_cols)

# Convert date columns (use format if known, e.g., '%Y-%m-%d')
df["StartDate"] = pd.to_datetime(df["StartDate"], errors="coerce")
df["ExitDate"] = pd.to_datetime(df["ExitDate"], errors="coerce")
df["DOB"] = pd.to_datetime(df["DOB"], errors="coerce")

# ------------------ 2. Average Tenure ------------------
today = datetime.today()
df["ExitDate_filled"] = df["ExitDate"].fillna(today)
df["Tenure_days"] = (df["ExitDate_filled"] - df["StartDate"]).dt.days
df["Tenure_years"] = df["Tenure_days"] / 365
avg_tenure = df["Tenure_years"].mean()

# ------------------ Map Performance Score to Numeric ------------------
perf_map = {
    "PIP": 0,
    "Needs Improvement": 1,
    "Fully Meets": 2,
    "Exceeds": 3
}
df["Performance_Score_Num"] = df["Performance Score"].map(perf_map)

# ------------------ 3. Performance Score Stats ------------------
perf_mean = df["Performance_Score_Num"].mean()
perf_median = df["Performance_Score_Num"].median()
perf_std = df["Performance_Score_Num"].std()

# ------------------ 4. Avg Current Employee Rating per Department ------------------
dep_rating = df.groupby("DepartmentType")["Current Employee Rating"].mean()
top_department = dep_rating.idxmax()

# ------------------ 5. Termination Analysis ------------------
termination_counts = df[df["ExitDate"].notna()].groupby("TerminationType")["EmpID"].count()
termination_reasons = df[df["ExitDate"].notna()]["TerminationDescription"].value_counts()

# ------------------ 6. Gender & Race ------------------
gender_counts = df["GenderCode"].value_counts()
race_percentage = df["RaceDesc"].value_counts(normalize=True) * 100

# ------------------ 7. Avg Performance Score per BusinessUnit ------------------
business_perf = df.groupby("BusinessUnit")["Performance_Score_Num"].mean().sort_values(ascending=False).head(3)

# ------------------ 8. Current vs Terminated Employees ------------------
status_counts = df["EmployeeStatus"].value_counts()

# ------------------ 9. Dept-wise Total Employees & Avg Performance ------------------
dept_summary = df.groupby("DepartmentType").agg(
    Total_Employees=("EmpID", "count"),
    Avg_Performance=("Performance_Score_Num", "mean")
)

# ------------------ 10. Performance by JobFunction & PayZone ------------------
job_pay_perf = df.groupby(["JobFunctionDescription", "PayZone"])["Performance_Score_Num"].mean()

# ------------------ 11. Performance by Supervisor ------------------
supervisor_perf = df.groupby("Supervisor")["Performance_Score_Num"].mean().sort_values(ascending=False).head(5)

# ------------------ 12. Marital Status ------------------
marital_percentage = df["MaritalDesc"].value_counts(normalize=True) * 100

# ------------------ 13. Age Analysis ------------------
# Calculate age only for non-missing DOBs
df["Age"] = (today - df["DOB"]).dt.days / 365
age_groups = pd.cut(df["Age"].dropna(), bins=range(20, 71, 10), labels=[f"{i}-{i+9}" for i in range(20, 70, 10)])
age_distribution = age_groups.value_counts(normalize=True) * 100

# ------------------ 14. Avg Performance by Location ------------------
location_perf = df.groupby("LocationCode")["Performance_Score_Num"].mean()
highest_location = location_perf.idxmax()
lowest_location = location_perf.idxmin()

# ------------------ Display Results ------------------
print("Average Tenure (Years):", round(avg_tenure, 2))
print("Performance Score - Mean:", round(perf_mean, 2), "Median:", perf_median, "Std:", round(perf_std, 2))
print("Top Department by Avg Rating:", top_department)
print("Termination Counts:\n", termination_counts)
print("Top Termination Reasons:\n", termination_reasons.head())
print("Gender Counts:\n", gender_counts)
print("Race %:\n", race_percentage)
print("Top 3 Business Units by Performance:\n", business_perf)
print("Employee Status Counts:\n", status_counts)
print("Department Summary:\n", dept_summary.head())
print("Job & PayZone Performance:\n", job_pay_perf.head())
print("Top 5 Supervisors by Performance:\n", supervisor_perf)
print("Marital Status %:\n", marital_percentage)
print("Age Distribution (10yr bins):\n", age_distribution)
print("Highest Location by Avg Performance:", highest_location)
print("Lowest Location by Avg Performance:", lowest_location)


Average Tenure (Years): 2.94
Performance Score - Mean: 2.0 Median: 2.0 Std: 0.55
Top Department by Avg Rating: Admin Offices
Termination Counts:
 TerminationType
Involuntary    388
Resignation    380
Retirement     377
Voluntary      388
Name: EmpID, dtype: int64
Top Termination Reasons:
 TerminationDescription
Me see picture nature degree benefit.        1
Tree choose manager ask prove three.         1
Price apply father morning population.       1
Next explain account stay.                   1
Scientist open would defense receive old.    1
Name: count, dtype: int64
Gender Counts:
 GenderCode
Female    1682
Male      1318
Name: count, dtype: int64
Race %:
 RaceDesc
Asian       20.966667
Black       20.600000
White       19.966667
Other       19.400000
Hispanic    19.066667
Name: proportion, dtype: float64
Top 3 Business Units by Performance:
 BusinessUnit
SVG    2.052632
TNS    2.033670
EW     2.029801
Name: Performance_Score_Num, dtype: float64
Employee Status Counts:
 EmployeeStatus

/var/folders/4f/0sk9h68s2ks8xp8fv_49mz5h0000gn/T/ipykernel_9370/1751871398.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["StartDate"] = pd.to_datetime(df["StartDate"], errors="coerce")
/var/folders/4f/0sk9h68s2ks8xp8fv_49mz5h0000gn/T/ipykernel_9370/1751871398.py:16: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["ExitDate"] = pd.to_datetime(df["ExitDate"], errors="coerce")
